In [40]:
import pandas as pd

def isNaN(x):
    return x != x

data1 = pd.read_csv("IDCJAC0001_070014_Data1.csv")
data1.to_csv("old_rain.csv")
data2 = pd.read_csv("IDCJAC0002_070014_Data1.csv")
data2.to_csv("old_temp.csv")
data3 = pd.read_csv("IDCJAC0001_070351_Data1.csv")
data3.to_csv("new_rain.csv")
data4 = pd.read_csv("IDCJAC0002_070351_Data1.csv")
data4.to_csv("new_temp.csv")

climate_data = pd.DataFrame(columns={"rainfall", "max_temp"})

for i, row in data1.iterrows():
    date = f"{row.Year}-{str(row.Month).zfill(2)}"
    date = pd.to_datetime(date, format="%Y-%m")
    climate_data.loc[date, "rainfall"] = row["Monthly Precipitation Total (millimetres)"]

for i, row in data2.iterrows():
    date = f"{row.Year}-{str(row.Month).zfill(2)}"
    date = pd.to_datetime(date, format="%Y-%m")
    climate_data.loc[date, "max_temp"] = row["Mean maximum temperature (°C)"]

for i, row in data3.iterrows():
    date = f"{row.Year}-{str(row.Month).zfill(2)}"
    date = pd.to_datetime(date, format="%Y-%m")
    climate_data.loc[date, "rainfall"] = row["Monthly Precipitation Total (millimetres)"]

for i, row in data4.iterrows():
    date = f"{row.Year}-{str(row.Month).zfill(2)}"
    date = pd.to_datetime(date, format="%Y-%m")
    climate_data.loc[date, "max_temp"] = row["Mean maximum temperature (°C)"]

new_data = climate_data[climate_data.index > pd.to_datetime("1970-12", format="%Y-%m")]
new_data.to_csv("climate_data.csv", index_label="date")

In [41]:
soi_data = pd.read_csv("soi_monthly.txt", header=None, names=["month", "soi"])
soi_data["month"] = pd.to_datetime(soi_data["month"], format="%Y%m")

soi_mean = []
for i in soi_data.index:
    soi_mean.append(round(soi_data.loc[i - 5:i].soi.mean(), 1))

soi_data["mean"] = soi_mean

soi_data[soi_data["month"] > pd.to_datetime("1970-12", format="%Y-%m")].to_csv("soi_data.csv", index=False)